# PAIG
PAIG offers tools designed to enhance the security and compliance of your AI applications. Whether you're using chatbots internally, incorporating AI services into your products, or utilizing automated tools to process tasks like customer support tickets or feedback, PAIG ensures that you maintain the highest standards of security and adherence to compliance. It's tailored for businesses that value a robust yet straightforward approach to AI application governance.


## 1. Install Dependencies
This will take several seconds, upto a minute. This installs PAIG library, PAIG Shield plugin library, OpenAI and Spacy


In [ ]:
!pip install -qqq paig_client openai paig-server --no-warn-conflicts
!python -m spacy download en_core_web_lg

## 2. Start PAIG Application

This will start PAIG application in the background on default port 4545.
You can run the below command to see all available setup options for PAIG application



```
paig --help
```

> **Tip:** Detailed PAIG application logs can be found in a directory called "logs"

In [ ]:
import subprocess

command = ["paig", "run"]

# Start the PAIG application in the background
# Note - Console logs are hidden using stdout parameter, please remove the stdout parameter to get console logs
process = subprocess.Popen(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print(f"Started PAIG application with PID {process.pid}")

## 3. Login to your PAIG application once ready

This will ask you to wait until PAIG application is ready for use. Once ready we will provide you with a link using which you can explore the PAIG user interface.

> **Tip:** You can log into PAIG application using following credentials:-
<br> username: admin
<br> password: welcome1

In [ ]:
import requests
import time
url = "http://127.0.0.1:4545/"

print('Please kindly wait until we confirm if your PAIG application is ready')
while True:
  try:
    response = requests.get(url, timeout=3)
    response.raise_for_status()
    break
  except:
    print('Server is not ready yet, please hang on...')
    time.sleep(3)
server_url = "http://127.0.0.1:4545"
print(f'Your PAIG application is ready now \nYou can access PAIG application on - {server_url}')
print("You can log into PAIG application using following credentials:-\n  username: admin \n  password: welcome1")

## 4. Download application config from PAIG Portal

This will download the pre-seeded application configuration file from PAIG application portal and copy it to the folder called "privacera"


In [ ]:
import requests
import shutil
import os
import json

login_url = "http://127.0.0.1:4545/account-service/api/login"
config_url = "http://127.0.0.1:4545/governance-service/api/ai/application/1/config/json/download"
username = "admin"
password = "welcome1"

session = requests.Session()

def download_and_upload_demo_app_config():
    login_payload = {
    "username": username,  
    "password": password
    }
    login_response = session.post(login_url, data=json.dumps(login_payload))
    
    if login_response.status_code == 200:
        response = session.get(config_url)
        if response.status_code == 200:
            with open("privacera-shield-PAIG-Demo-config.json", "w") as file:
                file.write(response.text)
            destination_folder = "privacera"
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)
            
            shutil.move("privacera-shield-PAIG-Demo-config.json", os.path.join(destination_folder, "privacera-shield-PAIG-Demo-config.json"))
            print("Application config file downloaded and moved to privacera folder successfully.")
        else:
            print(f"Failed to download Application config file. Status code: {response.status_code}")
    else:
        print("User authentication failed please check username and password")

download_and_upload_demo_app_config()

## 5. Configure OpenAI API Key
Setup your OpenAI key as an environment variable.Please paste and enter your key in given text box.

In [ ]:
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

## 6. Initialize the Privacera Shield library
This one line of code will initialize the Privacera Shield library so that it is ready to protect your LangChain application.

> **Tip:** If you get an error in this step, it could be because you have already run this step once. You can ignore the error and continue. You can also try restarting the Kernel and start from the beginning with the proper JSON file and OpenAI API key.

In [ ]:
from paig_client import client as paig_shield_client
from openai import OpenAI

# Set the OPENAI_API_KEY environment variable or set it here
openai_client = OpenAI()

paig_shield_client.setup(frameworks=[])


## 7. Run the LLMChain with your question
1. In this step, we are going to run the LLMChain with the prompt asked by a user named `testuser`.
2. Note how we are passing the username by creating a Privacera Shield context object.
3. Privacera Shield will intercept the prompt and the response coming from LLM for the `testuser` and run policies.
4. The PAIG service scans both the prompt and response text and runs security policies.
5. If the policies decide that the access is denied then an AccessControlException is thrown.

> **Note:** Here username used is `testuser` which is an external user. So the policies applied will be as per the public. For applying userspecific properties, create and use the user from PAIG portal `Account > User Management > User`.

In [ ]:
from paig_client import client as paig_shield_client
from paig_client.model import ConversationType
import paig_client.exception
import uuid

# Replace "testuser" with the user who is using the application. Or you can use the service username
user = "testuser"

# Generate a random UUID which will be used to bind a prompt with a reply
privacera_thread_id = str(uuid.uuid4())

try:
   with paig_shield_client.create_shield_context(username=user):
      prompt_text = "Who was the first President of USA and where did they live?"
      print(f"User Prompt: {prompt_text}")
      # Validate prompt with Privacera Shield
      updated_prompt_text = paig_shield_client.check_access(
         text=prompt_text,
         conversation_type=ConversationType.PROMPT,
         thread_id=privacera_thread_id
      )
      updated_prompt_text = updated_prompt_text[0].response_text
      print(f"User Prompt (After Privacera Shield): {updated_prompt_text}")
      if prompt_text != updated_prompt_text:
         print(f"Updated prompt text: {updated_prompt_text}")

      # Call LLM with updated prompt text
      PROMPT = f"""Use the following pieces of context to answer the question at the end.
        {updated_prompt_text}
        ANSWER:
        """

      response = openai_client.chat.completions.create(model="gpt-4", messages=[{"role": "user", "content": PROMPT}],
                                                       temperature=0)
      llm_response = response.choices[0].message.content
      print(f"LLM Response: {llm_response}")
      # Validate LLM response with Privacera Shield
      updated_reply_text = paig_shield_client.check_access(
         text=llm_response,
         conversation_type=ConversationType.REPLY,
         thread_id=privacera_thread_id
      )
      updated_reply_text = updated_reply_text[0].response_text
      print(f"LLM Response (After Privacera Shield): {updated_reply_text}")
except paig_client.exception.AccessControlException as e:
   # If access is denied, then this exception will be thrown. You can handle it accordingly.
   print(f"AccessControlException: {e}")

## 8. Review the access audits in PAIG portal under Security menu option
1. Now you can log in to the PAIG portal and check under `Security > Access Audits` section. You will see the audit record for the above run of your LangChain application.
2. You can click on the eye icon and see the details of the prompts sent by the application to the LLM and the responses coming from the LLM.
3. The default policy in PAIG for the application monitors the flow and tags the contents of the prompt and response.

In [ ]:
import webbrowser
audit_url = f'{server_url}#/audits_security'
print(f'To check access audits you can also open {audit_url}')
webbrowser.open(audit_url)